In [1]:
from google.colab import drive
drive.mount('/content/drive')




Mounted at /content/drive


In [2]:
# === 📦 INSTALL DEPENDENCIES ===
!pip install torch torchvision torchaudio
!pip install torchlibrosa librosa panns-inference kagglehub



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
# === 🧠 IMPORT LIBRARIES ===
import os
import torch
import librosa
import numpy as np
from panns_inference import models, AudioTagging
import kagglehub
import pandas as pd
from torch import nn
import torch.nn.functional as F
#from pytorch_utils import do_mixup, interpolate, pad_framewise_output
import warnings
warnings.filterwarnings("ignore")



In [4]:
'''!git clone https://github.com/qiuqiangkong/audioset_tagging_cnn.git
import sys
sys.path.append('/content/audioset_tagging_cnn') # Adjust the path if needed'''

"!git clone https://github.com/qiuqiangkong/audioset_tagging_cnn.git\nimport sys\nsys.path.append('/content/audioset_tagging_cnn') # Adjust the path if needed"

In [5]:
'''# === ⬇️ DOWNLOAD GENDER DATASET ===
dataset_path = kagglehub.dataset_download("murtadhanajim/gender-recognition-by-voiceoriginal")
print(f"\n📂 Dataset downloaded to: {dataset_path}")

'''


📂 Dataset downloaded to: /kaggle/input/gender-recognition-by-voiceoriginal


In [6]:
'''# === 🔍 FIND FIRST AUDIO FILE ===
audio_file = None
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith('.wav'):
            audio_file = os.path.join(root, file)
            print(f"\n🎧 Using audio file: {audio_file}")
            break
    if audio_file:
        break'''



🎧 Using audio file: /kaggle/input/gender-recognition-by-voiceoriginal/data/female/arctic_b0454.wav


In [7]:
'''import torch
import librosa
import numpy as np
from torch import nn
import pandas as pd
import torch.nn.functional as F
#from pytorch_utils import do_mixup, interpolate, pad_framewise_output
import warnings
warnings.filterwarnings("ignore")

def init_layer(layer):
    """Initialize a Linear or Convolutional layer."""
    nn.init.xavier_uniform_(layer.weight)
    if hasattr(layer, 'bias'):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)

def init_bn(bn):
    """Initialize a Batchnorm layer."""
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=in_channels,
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)

        self.conv2 = nn.Conv2d(in_channels=out_channels,
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)

        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()

    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

    def forward(self, input, pool_size=(2, 2), pool_type='avg'):
        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')

        return x

class AudioFeatureExtractor(nn.Module):
    def __init__(self, feature_dim=2048):
        super(AudioFeatureExtractor, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)

        # Batch normalization layers
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm2d(256)
        self.bn4 = nn.BatchNorm2d(512)

        # Adaptive pooling to ensure fixed output size
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))

        # Fully connected layers
        self.fc1 = nn.Linear(512 * 4 * 4, feature_dim)

        # Dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Apply convolutions with batch norm and ReLU
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.bn4(self.conv4(x)))

        # Apply adaptive pooling to get fixed size output
        x = self.adaptive_pool(x)

        # Flatten and apply fully connected layer
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc1(x)

        return x

def extract_features(audio_path, sr=22050, duration=10):
    """
    Extract features from an audio file

    Args:
        audio_path: Path to the audio file
        sr: Sample rate (default: 22050)
        duration: Duration in seconds to process (default: 10)

    Returns:
        2048-dimensional feature vector
    """
    # Load and preprocess audio
    audio, _ = librosa.load(audio_path, sr=sr)

    # Ensure consistent length
    if len(audio) > sr * duration:
        audio = audio[:sr * duration]
    else:
        audio = np.pad(audio, (0, sr * duration - len(audio)))

    # Extract mel spectrogram
    mel_spec = librosa.feature.melspectrogram(
        y=audio,
        sr=sr,
        n_mels=128,  # Increased number of mel bands
        n_fft=2048,
        hop_length=512
    )

    # Convert to log scale
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    # Normalize
    mel_spec_db = (mel_spec_db - mel_spec_db.mean()) / (mel_spec_db.std() + 1e-8)

    # Reshape for model input (batch_size, channels, height, width)
    mel_spec_db = np.expand_dims(mel_spec_db, axis=0)  # Add channel dimension
    mel_spec_db = np.expand_dims(mel_spec_db, axis=0)  # Add batch dimension

    # Convert to tensor
    mel_spec_tensor = torch.FloatTensor(mel_spec_db)

    # Initialize model
    model = AudioFeatureExtractor()
    model.eval()

    # Extract features
    with torch.no_grad():
        features = model(mel_spec_tensor)

    return features.numpy()

def main():
    # Example extraction
    audio_path = audio_file
    try:
        features = extract_features(audio_path)
        print(f"Extracted feature shape: {features.shape}")  # Should be (1, 2048)
        print(f"Extracted features: {features}")
    except Exception as e:
        print(f"Error processing audio file: {e}")

if __name__ == "__main__":
    main()
'''

Extracted feature shape: (1, 2048)
Extracted features: [[-0.04028089  0.01266167 -0.0225538  ... -0.00785626  0.00449951
  -0.01109302]]


In [ ]:


def init_layer(layer):
    """Initialize a Linear or Convolutional layer."""
    nn.init.xavier_uniform_(layer.weight)
    if hasattr(layer, 'bias'):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)

def init_bn(bn):
    """Initialize a Batchnorm layer."""
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=in_channels,
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)

        self.conv2 = nn.Conv2d(in_channels=out_channels,
                              out_channels=out_channels,
                              kernel_size=(3, 3), stride=(1, 1),
                              padding=(1, 1), bias=False)

        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()

    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

    def forward(self, input, pool_size=(2, 2), pool_type='avg'):
        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')

        return x

class AudioFeatureExtractor(nn.Module):
    def __init__(self, feature_dim=2048):
        super(AudioFeatureExtractor, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)

        # Batch normalization layers
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm2d(256)
        self.bn4 = nn.BatchNorm2d(512)

        # Adaptive pooling to ensure fixed output size
        self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))

        # Fully connected layers
        self.fc1 = nn.Linear(512 * 4 * 4, feature_dim)

        # Dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Apply convolutions with batch norm and ReLU
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.bn4(self.conv4(x)))

        # Apply adaptive pooling to get fixed size output
        x = self.adaptive_pool(x)

        # Flatten and apply fully connected layer
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc1(x)

        return x

def extract_features(audio_path, sr=22050, duration=10):
    """
    Extract features from an audio file

    Args:
        audio_path: Path to the audio file
        sr: Sample rate (default: 22050)
        duration: Duration in seconds to process (default: 10)

    Returns:
        2048-dimensional feature vector
    """
    # Load and preprocess audio
    audio, _ = librosa.load(audio_path, sr=sr)

    # Ensure consistent length
    if len(audio) > sr * duration:
        audio = audio[:sr * duration]
    else:
        audio = np.pad(audio, (0, sr * duration - len(audio)))

    # Extract mel spectrogram
    mel_spec = librosa.feature.melspectrogram(
        y=audio,
        sr=sr,
        n_mels=128,  # Increased number of mel bands
        n_fft=2048,
        hop_length=512
    )

    # Convert to log scale
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    # Normalize
    mel_spec_db = (mel_spec_db - mel_spec_db.mean()) / (mel_spec_db.std() + 1e-8)

    # Reshape for model input (batch_size, channels, height, width)
    mel_spec_db = np.expand_dims(mel_spec_db, axis=0)  # Add channel dimension
    mel_spec_db = np.expand_dims(mel_spec_db, axis=0)  # Add batch dimension

    # Convert to tensor
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    mel_spec_tensor = torch.FloatTensor(mel_spec_db).to(device)

    # Initialize model

    model = AudioFeatureExtractor().to(device)
    model.eval()

    # Extract features
    with torch.no_grad():
        features = model(mel_spec_tensor)#.cpu().numpy()

    return features.cpu().numpy()

def process_dataset(dataset_path, output_file='audio_features.csv'):
    """
    Processes all audio files in the dataset and stores features in a CSV file.

    Args:
        dataset_path: Path to the dataset directory.
        output_file: Path to the output CSV file.
    """

    features_list = []
    file_names = []

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith('.wav'):
                audio_path = os.path.join(root, file)
                try:
                    features = extract_features(audio_path)
                    features_list.append(features.flatten())  # Flatten to 1D array
                    file_names.append(file)
                    print(f"Processed: {file}")
                except Exception as e:
                    print(f"Error processing {file}: {e}")

    # Create a pandas DataFrame and save to CSV
    df = pd.DataFrame(features_list, index=file_names, columns=[f'feature_{i}' for i in range(features_list[0].shape[0])])
    df.to_csv(output_file, index_label='file_name')
    print(f"Features saved to: {output_file}")

if __name__ == "__main__":
    dataset_path = kagglehub.dataset_download("murtadhanajim/gender-recognition-by-voiceoriginal")
    process_dataset(dataset_path)

Streaming output truncated to the last 5000 lines.
Processed: arctic_a0342(5).wav
Processed: arctic_a0298(1).wav
Processed: arctic_b0020(3).wav
Processed: arctic_a0125(1).wav
Processed: arctic_a0146(5).wav
Processed: arctic_a0058(5).wav
Processed: arctic_a0388(4).wav
Processed: arctic_a0154.wav
Processed: arctic_a0300(6).wav
Processed: arctic_a0520(3).wav
Processed: arctic_a0367(5).wav
Processed: arctic_b0379.wav
Processed: arctic_a0409(1).wav
Processed: arctic_a0275(5).wav
Processed: arctic_a0177(6).wav
Processed: arctic_a0087(6).wav
Processed: arctic_b0234.wav
Processed: arctic_b0349(1).wav
Processed: arctic_a0231(7).wav
Processed: arctic_a0168(1).wav
Processed: arctic_b0482(3).wav
Processed: arctic_b0388.wav
Processed: arctic_a0342(4).wav
Processed: arctic_a0279(4).wav
Processed: arctic_a0186(7).wav
Processed: arctic_a0264(4).wav
Processed: arctic_b0048(1).wav
Processed: arctic_a0572.wav
Processed: arctic_a0244(4).wav
Processed: arctic_b0071(1).wav
Processed: arctic_a0006(4).wav
Pro